# Imports and Settings 

In [1]:
import sys
sys.path.append("../src")

import cv2
import dlib
import numpy as np
import pickle as pkl
from os.path import join
from icao_dataset import ICAODataset
from data_structures import Eye
from global_config import FOLDER_IMAGES, FOLDER_MRKS, IMAGE_SIZE

In [2]:
PATH_TO_ROOT = ".."
EYE_INDEXES = [36, 39, 42, 45]

FILE_SHAPE_PREDICTOR = "shape_predictor_68_face_landmarks.dat"
FILE_DATASET = join(PATH_TO_ROOT, "data/cropped_faces_eyes/dataset.pkl")
FOLDER_IMAGES = join(PATH_TO_ROOT, FOLDER_IMAGES)
FOLDER_MRKS = join(PATH_TO_ROOT, FOLDER_MRKS)

print(FILE_DATASET)
print(FOLDER_IMAGES)
print(FOLDER_MRKS)

..\data/cropped_faces_eyes/dataset.pkl
..\data/cropped_faces_eyes/images/
..\data/cropped_faces_eyes/ground_truth/


In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(FILE_SHAPE_PREDICTOR)

# Load Dataset 

In [4]:
dataset = ICAODataset()
dataset.load(FOLDER_IMAGES, FOLDER_MRKS, output_size=IMAGE_SIZE, interpolation=cv2.INTER_AREA)

5731 of 5731
5731 of 5731


In [5]:
%%time

new_dataset = ICAODataset()
n_files = len(dataset.image_files)

for i, (file, im, mrk) in enumerate(dataset):
    print(f"{i + 1} of {n_files}", end="\r")
    image = im.astype(np.uint8)
    rects = detector(image, 1)
    
    if len(rects) == 0:
        continue

    shape = predictor(image, rects[0])
    coordinates = np.array([(shape.part(i).x, shape.part(i).y) for i in EYE_INDEXES]).flatten()
    mrk.right_eye = Eye(*coordinates[:4])
    mrk.left_eye = Eye(*coordinates[4:])

    new_dataset.image_files.append(file)
    new_dataset.images.append(im)
    new_dataset.mrks.append(mrk)

Wall time: 1min 37s


In [6]:
FILE_OUTPUT = FILE_DATASET.replace(".pkl", "_dlib.pkl")

pkl.dump(new_dataset, open(FILE_OUTPUT, "wb"), protocol=4)